# SageMaker,MXNet,Gluonによるレコメンドシステムの実装
_**Making Video Recommendations Using Neural Networks and Embeddings**_

--- 

---

*This work is based on content from the [Cyrus Vahid's 2017 re:Invent Talk](https://github.com/cyrusmvahid/gluontutorials/blob/master/recommendations/MLPMF.ipynb)*


## Contents

1. [背景](#Background)
1. [セットアップ](#Setup)
1. [データ](#Data)
  1. [探索](#Explore)
  1. [クリーニング](#Clean)
  1. [前準備](#Prepare)
1. [ローカルでのTraining](#Train-Locally)
  1. [ネットワーク定義](#Define-Network)
  1. [パラメータセット](#Set-Parameters)
  1. [実行](#Execute)
1. [SageMakerでのTraining](#Train-with-SageMaker)
  1. [コード ラッピング](#Wrap-Code)
  1. [データの移動](#Move-Data)
  1. [提出](#Submit)
1. [ホスティング](#Host)
  1. [評価](#Evaluate)
1. [ラップアップ](#Wrap-up)

---

## 背景

In many ways, recommender systems were a catalyst for the current popularity of machine learning.  One of Amazon's earliest successes was the "Customers who bought this, also bought..." feature, while the million dollar Netflix Prize spurred research, raised public awareness, and inspired numerous other data science competitions.

多くの点で、レコメンドシステムは、昨今における機械学習ブームの火付け役となりました。Amazonの最も早期の成功の1つは、「この商品を買ったお客様はこんなものを見ています」というレコメンドシステムでした。100万ドルのNetflix賞はRecommendationの研究を促し、一般の人々の意識を高め、多くのデータサイエンス大会に影響を与えました。

Recommender systems can utilize a multitude of data sources and ML algorithms, and most combine various unsupervised, supervised, and reinforcement learning techniques into a holistic framework.  However, the core component is almost always a model which which predicts a user's rating (or purchase) for a certain item based on that user's historical ratings of similar items as well as the behavior of other similar users.  The minimal required dataset for this is a history of user item ratings.  In our case, we'll use 1 to 5 star ratings from over 2M Amazon customers on over 160K digital videos.  More details on this dataset can be found at its [AWS Public Datasets page](https://s3.amazonaws.com/amazon-reviews-pds/readme.html).

推薦システムは、多数のデータソースとMLアルゴリズムを利用することができ、多くの場合、教師あり、教師なし学習、強化学習等の、様々なテクニックを総合的なフレームワークに組み込みます。 ただし、コアコンポーネントは、ほとんどの場合、類似アイテムのそのユーザーの過去の評価および他の類似ユーザーの動作に基づいて、特定のアイテムのユーザーの評価（または購入）を予測するモデルです。 これに必要な最小限のデータセットは、ユーザーのアイテムレーティングの履歴です。 私たちの場合、160Kを超えるデジタルビデオで、2M以上のAmazonユーザーからの1から5の星評価を使用します。 このデータセットの詳細については、[AWS Public Datasets]ページ（https://s3.amazonaws.com/amazon-reviews-pds/readme.html）を参照してください。

Matrix factorization has been the cornerstone of most user-item prediction models.  This method starts with the large, sparse, user-item ratings in a single matrix, where users index the rows, and items index the columns.  It then seeks to find two lower-dimensional, dense matrices which, when multiplied together, preserve the information and relationships in the larger matrix.

![image](https://data-artisans.com/img/blog/factorization.svg)

行列分解は、ほとんどのユーザ項目予測モデルの基礎となっています。 このメソッドは、ユーザーが行を索引付けし、項目が列を索引付けする単一のマトリックス内に、大規模でスパースなユーザー項目の格付けから始まります。 次に、2つの低次元の密行列を求め、それらを掛け合わせると、より大きな行列の情報と関係が保存されます。

Matrix factorization has been extended and genarlized with deep learning and embeddings.  These techniques allows us to introduce non-linearities for enhanced performance and flexibility.  This notebook will fit a neural network-based model to generate recommendations for the Amazon video dataset.  It will start by exploring our data in the notebook and even training a model on a sample of the data.  Later we'll expand to the full dataset and fit our model using a SageMaker managed training cluster.  We'll then deploy to an endpoint and check our method.

行列分解は、深い学習と埋め込みで拡張され、genarlizedされました。 これらの技術により、非線形性を導入してパフォーマンスと柔軟性を向上させることができます。 このノートブックは、ニューラルネットワークベースのモデルに適合し、Amazonビデオデータセットの推奨を生成します。 ノートブックのデータを調べたり、サンプルのデータをモデル化したりすることから始めます。 その後、SageMaker管理トレーニングクラスターを使用して完全なデータセットに拡張し、モデルに適合させます。 その後、エンドポイントに展開してメソッドをチェックします。


---

## Setup

_This notebook was created and tested on an ml.p2.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `get_execution_role()` call with the appropriate full IAM role arn string(s).

このノートブックは、ml.p2.xlargeノートブックインスタンスで作成およびテストされました。

以下を指定して始めましょう。

トレーニングとモデルデータに使用するS3バケットとプレフィックス。 これは、ノートブックインスタンス、トレーニング、およびホスティングと同じ領域内にある必要があります。
IAMの役割は、トレーニングを行い、データへのアクセスをホストするために使用されます。 これらを作成する方法については、ドキュメントを参照してください。 ノートブックインスタンス、トレーニング、および/またはホスティングに複数のロールが必要な場合は、get_execution_role（）コールを適切な完全IAMロールのarnストリングに置き換えてください。

In [1]:
bucket = 'sagemaker-shokout'
prefix = 'sagemaker/DEMO-gluon-recsys'

import sagemaker
role = sagemaker.get_execution_role()

Now let's load the Python libraries we'll need for the remainder of this example notebook.

次に、このノートブックノートの残りの部分で必要となるPythonライブラリをロードしましょう。

In [2]:
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt

---
## Data

### Explore

Let's start by bringing in our dataset from an S3 public bucket.  As mentioned above, this contains 1 to 5 star ratings from over 2M Amazon customers on over 160K digital videos.  More details on this dataset can be found at its [AWS Public Datasets page](https://s3.amazonaws.com/amazon-reviews-pds/readme.html).

_Note, because this dataset is over a half gigabyte, the load from S3 may take ~10 minutes.  Also, since Amazon SageMaker Notebooks start with a 5GB persistent volume by default, and we don't need to keep this data on our instance for long, we'll bring it to the temporary volume (which has up to 20GB of storage)._

まず、S3の公開バケットからデータセットを取り込みましょう。 上記のとおり、これには160,000以上のデジタルビデオ上の2M以上のAmazonユーザーからの1から5の星評価が含まれています。 このデータセットの詳細については、[AWS Public Datasets]ページ（https://s3.amazonaws.com/amazon-reviews-pds/readme.html）を参照してください。

_注：このデータセットは半ギガバイトを超えているため、S3からの読み込みには約10分かかる場合があります。 また、Amazon SageMakerノートブックはデフォルトで5GBの永続的ボリュームから始まり、このデータをインスタンスに長く保存する必要はありません。一時的なボリューム（最大20GBのストレージ）に移動します。 _

In [3]:
!mkdir /tmp/recsys/
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz /tmp/recsys/

mkdir: cannot create directory ‘/tmp/recsys/’: File exists
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz to ../../../../../tmp/recsys/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz


In [4]:
#!aws s3 --no-sign-request cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz /tmp/recsys/

Let's read the data into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) so that we can begin to understand it.

*Note, we'll set `error_bad_lines=False` when reading the file in as there appear to be a very small number of records which would create a problem otherwise.*

In [5]:
df = pd.read_csv('/tmp/recsys/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

b'Skipping line 92523: expected 15 fields, saw 22\n'
b'Skipping line 343254: expected 15 fields, saw 22\n'
b'Skipping line 524626: expected 15 fields, saw 22\n'
b'Skipping line 623024: expected 15 fields, saw 22\n'
b'Skipping line 977412: expected 15 fields, saw 22\n'
b'Skipping line 1496867: expected 15 fields, saw 22\n'
b'Skipping line 1711638: expected 15 fields, saw 22\n'
b'Skipping line 1787213: expected 15 fields, saw 22\n'
b'Skipping line 2395306: expected 15 fields, saw 22\n'
b'Skipping line 2527690: expected 15 fields, saw 22\n'


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... ...,2015-08-31
1,US,30549954,R1IZHHS1MH3AQ4,B00KQD28OM,246219280,Vicious,Digital_Video_Download,5,0,0,N,Y,As always it seems that the best shows come fr...,As always it seems that the best shows come fr...,2015-08-31
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31
3,US,27072354,R7HOOYTVIB0DS,B008LOVIIK,239012694,Masterpiece: Inspector Lewis Season 5,Digital_Video_Download,5,0,0,N,Y,Five Stars,excellant this is what tv should be,2015-08-31
4,US,26939022,R1XQ2N5CDOZGNX,B0094LZMT0,535858974,On The Waterfront,Digital_Video_Download,5,0,0,N,Y,Brilliant film from beginning to end,Brilliant film from beginning to end. All of t...,2015-08-31


We can see this dataset includes information like:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

For this example, let's limit ourselves to `customer_id`, `product_id`, and `star_rating`.  Including additional features in our recommendation system could be beneficial, but would require substantial processing (particularly the text data) which would take us beyond the scope of this notebook.

*Note: we'll keep `product_title` on the dataset to help verify our recommendations later in the notebook, but it will not be used in algorithm training.*


---

このデータセットには次のような情報が含まれています。

- `marketplace`：2文字の国コード（この場合はすべて「US」）。
- `customer_id`：単一の著者によって書かれたレビューを集約するために使用できるランダムな識別子。
- `review_id`：レビューの一意のIDです。
- `product_id`：Amazon標準識別番号（ASIN）です。 `http://www.amazon.com/dp/ <ASIN>`は製品の詳細ページにリンクしています。
- `product_parent`：そのASINの親です。複数のASIN（同じ製品のカラーバリエーションまたはフォーマットバリエーション）を1つの親の親にロールアップすることができます。
- `product_title`：製品のタイトルの説明。
- 'product_category`：レビューをグループ化するために使用できる幅広い製品カテゴリ（この場合はデジタルビデオ）。
- `star_rating`：レビューの評価（1〜5つ星）
- `helpful_votes`：レビューの有益な投票数。
- `total_votes`：レビューが受け取った総投票数。
- `vine`：レビューは[Vine]（https://www.amazon.com/gp/vine/help）プログラムの一部として書かれましたか？
- `verified_purchase`：確認された購入からのレビューですか？
- `review_headline`：レビュー自体のタイトル。
- `review_body`：レビューのテキスト。
- `review_date`：レビューが書かれた日付。

この例では、 `customer_id`、` product_id`、 `star_rating`に限定しましょう。推奨システムに追加機能を組み込むことは有益ですが、このノートブックの範囲を超えてしまうかなりの処理（特にテキストデータ）が必要になります。

*ノート：後でノートブックの推奨事項を確認するために `product_title`をデータセットに保存しますが、アルゴリズムトレーニングには使用されません。

In [6]:
df = df[['customer_id', 'product_id', 'star_rating', 'product_title']]

Because most people haven't seen most videos, and people rate fewer videos than we actually watch, we'd expect our data to be sparse.  Our algorithm should work well with this sparse problem in general, but we may still want to clean out some of the long tail.  Let's look at some basic percentiles to confirm.

ほとんどの人はほとんどの動画を見たことがなく、実際に見るよりも動画の割合が低いため、データが不足していると思われます。 私たちのアルゴリズムはこの一般的なスパースな問題にはうまくいくはずですが、まだ長いテールのいくつかを取り除きたいかもしれません。 確認するためにいくつかの基本的なパーセンタイルを見てみましょう。

In [7]:
customers = df['customer_id'].value_counts()
products = df['product_id'].value_counts()

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1]
print('customers\n', customers.quantile(quantiles))
print('products\n', products.quantile(quantiles))

customers
 0.00       1.0
0.01       1.0
0.02       1.0
0.03       1.0
0.04       1.0
0.05       1.0
0.10       1.0
0.25       1.0
0.50       1.0
0.75       2.0
0.90       4.0
0.95       5.0
0.96       6.0
0.97       7.0
0.98       9.0
0.99      13.0
1.00    2704.0
Name: customer_id, dtype: float64
products
 0.00        1.00
0.01        1.00
0.02        1.00
0.03        1.00
0.04        1.00
0.05        1.00
0.10        1.00
0.25        1.00
0.50        3.00
0.75        9.00
0.90       31.00
0.95       73.00
0.96       95.00
0.97      130.00
0.98      199.00
0.99      386.67
1.00    32790.00
Name: product_id, dtype: float64


As we can see, only about 5% of customers have rated 5 or more videos, and only 25% of videos have been rated by 9+ customers.

ご覧のように、5％以上の動画を評価したのは約5％に過ぎず、9％以上のお客様が動画の25％しか評価していません。

### Clean

Let's filter out this long tail.

この長い尾を除外しましょう。

In [8]:
customers = customers[customers >= 5]
products = products[products >= 10]

reduced_df = df.merge(pd.DataFrame({'customer_id': customers.index})).merge(pd.DataFrame({'product_id': products.index}))

Now, we'll recreate our customer and product lists since there are customers with more than 5 reviews, but all of their reviews are on products with less than 5 reviews (and vice versa).

今度は、5つ以上のレビューを持つ顧客が存在するため、顧客リストと商品リストを再作成しますが、すべてのレビューは5件未満のレビュー（およびその逆）の商品にあります。

In [9]:
customers = reduced_df['customer_id'].value_counts()
products = reduced_df['product_id'].value_counts()

Next, we'll number each user and item, giving them their own sequential index.  This will allow us to hold the information in a sparse format where the sequential indices indicate the row and column in our ratings matrix.

次に、それぞれのユーザーとアイテムに番号を付け、それぞれに独自の順次インデックスを付けます。 これにより、私たちは、連続したインデックスが評価マトリックスの行と列を示す疎フォーマットで情報を保持することができます。

In [10]:
customer_index = pd.DataFrame({'customer_id': customers.index, 'user': np.arange(customers.shape[0])})
product_index = pd.DataFrame({'product_id': products.index, 
                              'item': np.arange(products.shape[0])})

reduced_df = reduced_df.merge(customer_index).merge(product_index)
reduced_df.head()

,customer_id,product_id,star_rating,product_title,user,item
0,27072354,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,10463,107
1,16030865,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,489,107
2,44025160,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,32100,107
3,18602179,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,2237,107
4,14424972,B008LOVIIK,5,Masterpiece: Inspector Lewis Season 5,32340,107


### 準備

Let's start by splitting in training and test sets.  This will allow us to estimate the model's accuracy on videos our customers rated, but wasn't included in our training.

トレーニングとテストセットを分割して始めましょう。 これにより、お客様が評価した動画のモデルの精度を見積もることができますが、トレーニングには含まれませんでした。

In [11]:
test_df = reduced_df.groupby('customer_id').last().reset_index()

train_df = reduced_df.merge(test_df[['customer_id', 'product_id']], 
                            on=['customer_id', 'product_id'], 
                            how='outer', 
                            indicator=True)
train_df = train_df[(train_df['_merge'] == 'left_only')]

Now we'll prepare for MXNet.  The data is sparse, so we'll first create a sparse matrix class that takes in our features (user and item indices) and our label (the star rating for that user-item combination).  This actually represents our data as a long list of (row index, column index, rating) values, rather than the large sparse user-item matrix shown above.  However, it doesn't change the mathematics of our model.

MXNetの準備をします。 データは疎なので、最初に私たちの特徴（ユーザーとアイテムのインデックス）と私たちのラベル（そのユーザーアイテムの組み合わせの星の評価）を取り入れるスパース行列クラスを作成します。 これは実際に私たちのデータを上記の大きな疎なユーザ項目マトリックスではなく長いリスト（行インデックス、列インデックス、評価）の値として表しています。 ただし、モデルの数学は変更されません。

In [12]:
class SparseMatrixDataset(gluon.data.Dataset):
    def __init__(self, data, label):
        assert data.shape[0] == len(label)
        self.data = data
        self.label = label
        if isinstance(label, ndarray.NDArray) and len(label.shape) == 1:
            self._label = label.asnumpy()
        else:
            self._label = label       
        
    def __getitem__(self, idx):
        return self.data[idx, 0], self.data[idx, 1], self.label[idx]
    
    def __len__(self):
        return self.data.shape[0]        

Now, we can convert our Pandas DataFrames into MXNet NDArrays, use those to create a member of the SparseMatrixDataset class, and add that to an MXNet Data Iterator.  This process is the same for both test and control.

今度は、Pandas DataFramesをMXNet NDArraysに変換し、SparseMatrixDatasetクラスのメンバーを作成してMXNet Data Iteratorに追加します。 このプロセスは、テストとコントロールの両方で同じです。

In [13]:
batch_size = 40

train_iter = gluon.data.DataLoader(SparseMatrixDataset(nd.array(train_df[['user', 'item']].values, dtype=np.float32), 
                                                       nd.array(train_df['star_rating'].values, dtype=np.float32)), 
                                   shuffle=True,
                                   batch_size=batch_size)
test_iter = gluon.data.DataLoader(SparseMatrixDataset(nd.array(test_df[['user', 'item']].values, dtype=np.float32), 
                                                      nd.array(test_df['star_rating'].values, dtype=np.float32)),
                                  shuffle=True,
                                  batch_size=batch_size)

---

## ローカルでTrainingする

### Define Network

Let's start by defining the neural network version of our matrix factorization task.  In this case, our network is quite simple.  The main components are:
- [Embeddings](https://mxnet.incubator.apache.org/api/python/gluon/nn.html#mxnet.gluon.nn.Embedding) which turn our indexes into dense vectors of fixed size.  In this case, 64.
- [Dense layers](https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dense) with ReLU activation.  Each dense layer has the same number of units as our number of embeddings.  Our ReLU activation here also adds some non-linearity to our matrix factorization.
- [Dropout layers](https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dropout) which can be used to prevent over-fitting.
- Matrix multiplication of our user matrix and our item matrix to create an estimate of our rating matrix.


マトリックス分解タスクのニューラルネットワーク版を定義することから始めましょう。 この場合、私たちのネットワークは非常に簡単です。 主なコンポーネントは次のとおりです。
- [Embeddings]（https://mxnet.incubator.apache.org/api/python/gluon/nn.html#mxnet.gluon.nn.Embedding）インデックスを固定サイズの密ベクトルに変換します。 この場合、64。
- ReLUを有効にした[Dense layers]（https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dense） 各高密度層は、我々の埋め込み数と同じ数のユニットを有する。 ここでのReLUの活性化は、行列分解に非線形性を追加します。
- 過剰嵌合を防止するために使用できる[ドロップアウト層]（https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dropout）。
- ユーザマトリクスとアイテムマトリクスのマトリクス乗算により、評価マトリクスの見積もりを作成します。

In [ ]:
class MFBlock(gluon.HybridBlock):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFBlock, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            self.dropout = gluon.nn.Dropout(dropout_p)
            self.dense = gluon.nn.Dense(num_emb, activation='relu')
            
    def hybrid_forward(self, F, users, items):
        a = self.user_embeddings(users)
        a = self.dense(a)
        
        b = self.item_embeddings(items)
        b = self.dense(b)

        predictions = self.dropout(a) * self.dropout(b)      
        predictions = F.sum(predictions, axis=1)
        return predictions

In [ ]:
num_embeddings = 64

net = MFBlock(max_users=customer_index.shape[0], 
              max_items=product_index.shape[0],
              num_emb=num_embeddings,
              dropout_p=0.)
net.collect_params()

### パラメータのセット

Let's initialize network weights and set our optimization parameters.

ネットワークの重みを初期化し、最適化パラメータをセットしましょう。

In [ ]:
# Initialize network parameters
# ctx = mx.gpu()
ctx = mx.cpu()
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24),
                                ctx=ctx,
                                force_reinit=True)
net.hybridize()

# Set optimization parameters
opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

trainer = gluon.Trainer(net.collect_params(),
                        opt,
                        {'learning_rate': lr,
                         'wd': wd,
                         'momentum': momentum})

### Execute

Let's define a function to carry out the training of our neural network.

In [ ]:
def execute(train_iter, test_iter, net, epochs, ctx):
    loss_function = gluon.loss.L2Loss()
    for e in range(epochs):
        print("epoch: {}".format(e))
        for i, (user, item, label) in enumerate(train_iter):
            try:
                user = user.as_in_context(ctx).reshape((batch_size,))
                item = item.as_in_context(ctx).reshape((batch_size,))
                label = label.as_in_context(ctx).reshape((batch_size,))
                with mx.autograd.record():
                    output = net(user, item)               
                    loss = loss_function(output, label)
                loss.backward()
                trainer.step(batch_size)
            except:
                pass
        print("EPOCH {}: MSE ON TRAINING and TEST: {}. {}".format(e,
                                                                   eval_net(train_iter, net, ctx, loss_function),
                                                                   eval_net(test_iter, net, ctx, loss_function)))
    print("end of training")
    return net

Let's also define a function which evaluates our network on a given dataset.  This is called by our `execute` function above to provide mean squared error values on our training and test datasets.

In [ ]:
def eval_net(data, net, ctx, loss_function):
    acc = MSE()
    for i, (user, item, label) in enumerate(data):
        try:
            user = user.as_in_context(ctx).reshape((batch_size,))
            item = item.as_in_context(ctx).reshape((batch_size,))
            label = label.as_in_context(ctx).reshape((batch_size, 1))
            predictions = net(user, item).reshape((batch_size, 1))
            acc.update(preds=[predictions], labels=[label])
        except:
            pass
    return acc.get()[1]

Now, let's train for a few epochs.

In [ ]:
%%time

epochs = 3

trained_net = execute(train_iter, test_iter, net, epochs, ctx)

#### Early Validation

We can see our training error going down, but our validation accuracy bounces around a bit.  Let's check how our model is predicting for an individual user.  We could pick randomly, but for this case, let's try user #6.

In [ ]:
product_index['u6_predictions'] = trained_net(nd.array([6] * product_index.shape[0]).as_in_context(ctx), 
                                              nd.array(product_index['item'].values).as_in_context(ctx)).asnumpy()
product_index.sort_values('u6_predictions', ascending=False)

Now let's compare this to the predictions for another user (we'll try user #7).

In [ ]:
product_index['u7_predictions'] = trained_net(nd.array([7] * product_index.shape[0]).as_in_context(ctx), 
                                              nd.array(product_index['item'].values).as_in_context(ctx)).asnumpy()
product_index.sort_values('u7_predictions', ascending=False)

The predicted ratings are different between the two users, but the same top (and bottom) items for user #6 appear for #7 as well.  Let's look at the correlation across the full set of 38K items to see if this relationship holds.

In [ ]:
product_index[['u6_predictions', 'u7_predictions']].plot.scatter('u6_predictions', 'u7_predictions')
plt.show()

We can see that this correlation is nearly perfect.  Essentially the average rating of items dominates across users and we'll recommend the same well-reviewed items to everyone.  As it turns out, we can add more embeddings and this relationship will go away since we're better able to capture differential preferences across users.

However, with just a 64 dimensional embedding, it took 40 minutes to run just 3 epochs.  If we ran this outside of our Notebook Instance we could run larger jobs and move on to other work would improve productivity.

---

## Train with SageMaker

Now that we've trained on this smaller dataset, we can expand training in SageMaker's distributed, managed training environment.

### Wrap Code

To use SageMaker's pre-built MXNet container, we'll need to wrap our code from above into a Python script.  There's a great deal of flexibility in using SageMaker's pre-built containers, and detailed documentation can be found [here](https://github.com/aws/sagemaker-python-sdk#mxnet-sagemaker-estimators), but for our example, it consisted of:
1. Wrapping all data preparation into a `prepare_train_data` function (we could name this whatever we like)
1. Copying and pasting classes and functions from above word-for-word
1. Defining a `train` function that:
  1. Adds a bit of new code to pick up the input TSV dataset on the SageMaker Training cluster
  1. Takes in a dict of hyperparameters (which we specified as globals above)
  1. Creates the net and executes training
  
---

この相関関係はほぼ完璧であることがわかります。基本的にはアイテムの平均評価がユーザー全体を支配しており、全員によくレビューされたアイテムを推奨します。結果として、より多くの埋め込みを追加することができます。この関係は、ユーザー間の差別的な嗜好をよりよく捕えることができます。

しかし、わずか64次元の埋め込みでは、わずか3エポックを実行するのに40分かかりました。これをNotebook Instanceの外で実行した場合、より大きな仕事を実行して他の仕事に進むことができ、生産性が向上します。

---

## SageMakerでトレーニングする

この小さなデータセットで訓練したので、SageMakerの分散管理されたトレーニング環境でトレーニングを拡張することができます。

###ラップコード

SageMakerのあらかじめ構築されたMXNetコンテナを使用するには、上のコードをPythonスクリプトにラップする必要があります。 SageMakerの事前構築されたコンテナの使用には多大な柔軟性があり、詳細な文書はこちら（https://github.com/aws/sagemaker-python-sdk#mxnet-sagemaker-estimators）にありますが、例：
1.すべてのデータ準備を `prepare_train_data`関数にラップします（私たちが好きなように名前をつけることができます）
1.単語から単語までのクラスと関数のコピーと貼り付け
1.次のような `train`関数を定義する：
  1. SageMaker Trainingクラスタで入力TSVデータセットをピックアップするための新しいコードを少し追加します
  1.ハイパーパラメータのdictを取る（これを上記のグローバルとして指定）
  1.ネットを作成し、トレーニングを実行する

In [14]:
!cat recommender.py

import logging
import json
import time
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE
import numpy as np

os.system('pip install pandas')
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

#########
# Globals
#########

batch_size = 40


##########
# Training
##########

def train(channel_input_dirs, hyperparameters, hosts, num_gpus, **kwargs):
    
    # get data
    training_dir = channel_input_dirs['train']
    train_iter, test_iter, customer_index, product_index = prepare_train_data(training_dir)
    
    # get hyperparameters
    num_embeddings = hyperparameters.get('num_embeddings', 64)
    opt = hyperparameters.get('opt', 'sgd')
    lr = hyperparameters.get('lr', 0.02)
    momentum = hyperparameters.get('momentum', 0.9)
    wd = hyperparameters.get('wd', 0.)
    epochs = hyperparameters.get('epochs', 5)

    # define net
    ctx = mx.gpu()

    net = MFBlock(max_users=customer_index.shape[0], 
                  max_item

### Test Locally

Now we can test our train function locally.  This helps ensure we don't have any bugs before submitting our code to SageMaker's pre-built MXNet container.

In [ ]:
# %%time

# import recommender

# local_test_net, local_customer_index, local_product_index = recommender.train(
#     {'train': '/tmp/recsys/'}, 
#     {'num_embeddings': 64, 
#      'opt': 'sgd', 
#      'lr': 0.02, 
#      'momentum': 0.9, 
#      'wd': 0.,
#      'epochs': 3},
#     ['local'],
#     1)

### Move Data

Holding our data in memory works fine when we're interactively exploring a sample of data, but for larger, longer running processes, we'd prfer to run them in the background with SageMaker Training.  To do this, let's move the dataset to S3 so that it can be picked up by SageMaker training.  This is perfect for use cases like periodic re-training, expanding to a larger dataset, or moving production workloads to larger hardware.

In [15]:
boto3.client('s3').copy({'Bucket': 'amazon-reviews-pds', 
                         'Key': 'tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'},
                        bucket,
                        prefix + '/train/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz')

In [16]:
# Set optimization parameters
opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

### Submit

Now, we can create an MXNet estimator from the SageMaker Python SDK.  To do so, we need to pass in:
1. Instance type and count for our SageMaker Training cluster.  SageMaker's MXNet containers support distributed GPU training, so we could easily set this to multiple ml.p2 or ml.p3 instances if we wanted.
  - *Note, this would require some changes to our recommender.py script as we would need to setup the context an key value store properly, as well as determining if and how to distribute the training data.*
1. An S3 path for out model artifacts and a role with access to S3 input and output paths.
1. Hyperparameters for our neural network.  Since with a 64 dimensional embedding, our recommendations reverted too closely to the mean, let's increase this by an order of magnitude when we train outside of our local instance.  We'll also increase the epochs to see how our accuracy evolves over time. We'll leave all other hyperparameters the same.

Once we use `.fit()` this creates a SageMaker Training Job that spins up instances, loads the appropriate packages and data, runs our `train` function from `recommender.py`, wraps up and saves model artifacts to S3, and finishes by tearing down the cluster.

In [ ]:
m = MXNet('recommender.py', 
          py_version='py3',
          role=role, 
          train_instance_count=1, 
          train_instance_type="ml.p3.8xlarge",
          output_path='s3://{}/{}/output'.format(bucket, prefix),
          hyperparameters={'num_embeddings': 512, 
                           'opt': opt, 
                           'lr': lr, 
                           'momentum': momentum, 
                           'wd': wd,
                           'epochs': 10})

m.fit({'train': 's3://{}/{}/train/'.format(bucket, prefix)})

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-314676777416
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-10-16-23-51-48-473


2018-10-16 23:51:48 Starting - Starting the training job...
Launching requested ML instances......
Preparing the instances for training......
2018-10-16 23:53:55 Downloading - Downloading input data
2018-10-16 23:54:08 Training - Downloading the training image...
Training image download completed. Training in progress..
2018-10-16 23:54:47,523 INFO - root - running container entrypoint
2018-10-16 23:54:47,523 INFO - root - starting train task
2018-10-16 23:54:47,580 INFO - container_support.training - Training starting
2018-10-16 23:54:48,880 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/mxnet#updating-your-mxnet-training-script.#033
2018-10-16 23:54:48,920 INFO - mxnet_

---

## Host

Now that we've trained our model, deploying it to a real-time, production endpoint is easy.

---

＃＃ ホスト

モデルを訓練したので、リアルタイムのプロダクションエンドポイントに導入するのは簡単です。

In [ ]:
predictor = m.deploy(initial_instance_count=1, 
                     instance_type='ml.m4.xlarge')
predictor.serializer = None

Now that we have an endpoint, let's test it out.  We'll predict user #6's ratings for the top and bottom ASINs from our local model.

*This could be done by sending HTTP POST requests from a separate web service, but to keep things easy, we'll just use the `.predict()` method from the SageMaker Python SDK.*

エンドポイントが完成したので、テストしましょう。 私たちは、ローカルモデルからトップ6と下部ASINのユーザー＃6の評価を予測します。

*これは別のWebサービスからのHTTP POSTリクエストを送信することで行うことができますが、簡単にするために、SageMaker Python SDKの `.predict（）`メソッドを使用します。

In [ ]:
predictor.predict(json.dumps({'customer_id': customer_index[customer_index['user'] == 6]['customer_id'].values.tolist(), 
                              'product_id': ['B00KH1O9HW', 'B00M5KODWO']}))

*Note, some of our predictions are actually greater than 5, which is to be expected as we didn't do anything special to account for ratings being capped at that value.  Since we are only looking to ranking by predicted rating, this won't create problems for our specific use case.*

### Evaluate

Let's start by calculating a naive baseline to approximate how well our model is doing.  The simplest estimate would be to assume every user item rating is just the average rating over all ratings.

*Note, we could do better by using each individual video's average, however, in this case it doesn't really matter as the same conclusions would hold.*

In [ ]:
print('Naive MSE:', np.mean((test_df['star_rating'] - np.mean(train_df['star_rating'])) ** 2))

Now, we'll calculate predictions for our test dataset.

*Note, this will align closely to our CloudWatch output above, but may differ slightly due to skipping partial mini-batches in our eval_net function.*

In [ ]:
test_preds = []
for array in np.array_split(test_df[['customer_id', 'product_id']].values, 40):
    test_preds += predictor.predict(json.dumps({'customer_id': array[:, 0].tolist(), 
                                                'product_id': array[:, 1].tolist()}))

test_preds = np.array(test_preds)
print('MSE:', np.mean((test_df['star_rating'] - test_preds) ** 2))

We can see that our neural network and embedding model produces substantially better results (~1.27 vs 1.65 on mean square error).

For recommender systems, subjective accuracy also matters.  Let's get some recommendations for a random user to see if they make intuitive sense.

In [ ]:
reduced_df[reduced_df['user'] == 6].sort_values(['star_rating', 'item'], ascending=[False, True])

As we can see, user #6 seems to like sprawling dramamtic television series and sci-fi, but they dislike silly comedies.

Now we'll loop through and predict user #6's ratings for every common video in the catalog, to see which ones we'd recommend and which ones we wouldn't.

In [ ]:
predictions = []
for array in np.array_split(product_index['product_id'].values, 40):
    predictions += predictor.predict(json.dumps({'customer_id': customer_index[customer_index['user'] == 6]['customer_id'].values.tolist() * array.shape[0], 
                                                 'product_id': array.tolist()}))

predictions = pd.DataFrame({'product_id': product_index['product_id'],
                            'prediction': predictions})

In [ ]:
titles = reduced_df.groupby('product_id')['product_title'].last().reset_index()
predictions_titles = predictions.merge(titles)
predictions_titles.sort_values(['prediction', 'product_id'], ascending=[False, True])

Indeed, our predicted highly rated shows have some well-reviewed TV dramas and some sci-fi.  Meanwhile, our bottom rated shows include goofball comedies.

*Note, because of random initialization in the weights, results on subsequent runs may differ slightly.*

Let's confirm that we no longer have almost perfect correlation in recommendations with user #7.

In [ ]:
predictions_user7 = []
for array in np.array_split(product_index['product_id'].values, 40):
    predictions_user7 += predictor.predict(json.dumps({'customer_id': customer_index[customer_index['user'] == 7]['customer_id'].values.tolist() * array.shape[0], 
                                                       'product_id': array.tolist()}))
plt.scatter(predictions['prediction'], np.array(predictions_user7))
plt.show()

---

## Wrap-up

In this example, we developed a deep learning model to predict customer ratings.  This could serve as the foundation of a recommender system in a variety of use cases.  However, there are many ways in which it could be improved.  For example we did very little with:
- hyperparameter tuning
- controlling for overfitting (early stopping, dropout, etc.)
- testing whether binarizing our target variable would improve results
- including other information sources (video genres, historical ratings, time of review)
- adjusting our threshold for user and item inclusion 

In addition to improving the model, we could improve the engineering by:
- Setting the context and key value store up for distributed training
- Fine tuning our data ingestion (e.g. num_workers on our data iterators) to ensure we're fully utilizing our GPU
- Thinking about how pre-processing would need to change as datasets scale beyond a single machine

Beyond that, recommenders are a very active area of research and techniques from active learning, reinforcement learning, segmentation, ensembling, and more should be investigated to deliver well-rounded recommendations.

### Clean-up (optional)

Let's finish by deleting our endpoint to avoid stray hosting charges.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)